In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier, RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd
from imblearn.over_sampling import SMOTE

## Подцепляем датасет

In [2]:
data = pd.read_csv('files/train.csv', sep=',')

## Обработка данных

In [3]:
data = data.drop(columns=['Product ID']) # Удаляем

In [4]:
# data['Type'] = data['Type'].astype('category').cat.codes

# le = LabelEncoder()
# data['Type'] = le.fit_transform(data['Type'])

In [5]:
# Применение get_dummies для столбца 'Type' из библиотеки pandas (по факту делает тоже самое, но быстрее чем One-Hot Encoding)
data = pd.get_dummies(data, columns=['Type'], prefix='Type')

In [6]:
# Применение One-Hot Encoding для столбца 'Type' из sklearn
# ohe = OneHotEncoder(sparse_output=False)  

# type_encoded = ohe.fit_transform(data[['Type']])  #
# type_encoded_df = pd.DataFrame(type_encoded, columns=ohe.get_feature_names_out(['Type']))

# data = pd.concat([data.drop(columns=['Type']), type_encoded_df], axis=1)

In [7]:
X = data.drop('Machine failure', axis=1)
y = data['Machine failure']

X.columns = X.columns.astype(str)
X.columns = X.columns.str.replace(r'[\[\]<]', '', regex=True)
X.columns = X.columns.str.replace(' ', '_')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## SMOTE для балансировки классов

Без него не обойтишь у нас сильный перекос в 0
Можно убрать если будем использовать сбалансированный датасет, и нужно оставить если будут "боевые" данные

In [8]:
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [9]:
print("До SMOTE:", y_train.value_counts())
print("После SMOTE:", pd.Series(y_train_smote).value_counts())

До SMOTE: Machine failure
0    107425
1      1718
Name: count, dtype: int64
После SMOTE: Machine failure
0    107425
1    107425
Name: count, dtype: int64


## Создание модели Stacking

In [10]:
# Параметры для модели

lgbm_params = {
    'objective': 'binary',        # Задача бинарной классификации.
    'metric': 'binary_logloss',   # Метрика для оценки ошибки - логарифмическая функция потерь (logloss).
    'num_leaves': 31,             # Максимальное количество листьев в одном дереве. Больше листьев — более сложная модель.
    'learning_rate': 0.05,        # Скорость обучения. Маленькие значения требуют больше итераций (n_estimators).
    'n_estimators': 500,          # Количество деревьев (итераций бустинга).
    'random_state': 42,           # Фиксируем случайное состояние для воспроизводимости результата.
    'force_row_wise': True,       # Принудительная многопоточность по строкам
    'verbose': -1                 # Отключение информационных сообщений
}


In [11]:
stacking_model = StackingClassifier(
    estimators=[
        ('rf', RandomForestClassifier(random_state=42)),
        ('xgb', XGBClassifier(eval_metric='logloss', random_state=42)),
        ('lgbm', LGBMClassifier(**lgbm_params))  # Используем lgbm_params здесь
    ],
    final_estimator=LogisticRegression(random_state=42)
)

## Обучение модели

In [12]:
stacking_model.fit(X_train_smote, y_train_smote)

StackingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                               ('xgb',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='logloss',
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importa...
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=None, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...)),
                               ('lgbm',
                                LGBMClassifier(force_row_wise=True,
                                               learning_rate=0.05,
                                               metric='binary_logloss',
                                               n_estimators=500,
                                               objective='binary',
                                               random_state=42, verbose=-1))],
                   final_estimator=LogisticRegression(random_state=42))

## Прогнозирование

In [13]:
y_pred_stacking = stacking_model.predict(X_test)

## Оценка модели

In [14]:
stacking_accuracy = accuracy_score(y_test, y_pred_stacking)
stacking_report = classification_report(y_test, y_pred_stacking, output_dict=True)

## Сохранение результатов

In [15]:
from tabulate import tabulate  # Вот эту строчку можно убрать, она для красоты вывода

results = []
results.append({
    "Модель": "Stacking", 
    "Точность (Accuracy)": stacking_accuracy,  
    "Точность (Precision)": stacking_report['weighted avg']['precision'],  
    "Полнота (Recall)": stacking_report['weighted avg']['recall'],  
    "F1-метрика (F1-Score)": stacking_report['weighted avg']['f1-score']  
})
results_df = pd.DataFrame(results)

print("Метрики производительности модели:")
print(tabulate(results, headers="keys", tablefmt="grid"))

Метрики производительности модели:
+----------+-----------------------+------------------------+--------------------+-------------------------+
| Модель   |   Точность (Accuracy) |   Точность (Precision) |   Полнота (Recall) |   F1-метрика (F1-Score) |
+==========+=======================+========================+====================+=========================+
| Stacking |              0.991937 |               0.992425 |           0.991937 |                0.992147 |
+----------+-----------------------+------------------------+--------------------+-------------------------+


In [ ]:
from joblib import dump

# Сохранение модели
dump(stacking_model, 'trained_stacking_model.joblib')  # Сохраняем модель в файл
print("Модель успешно сохранена в 'trained_stacking_model.joblib'.")

Модель успешно сохранена в 'trained_stacking_model.jobli'.
